In [46]:
%load_ext autoreload
%autoreload 2
import sys
import os

import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from train import setup_train_args, build, train_epoch, test
datasets = ["Kvasir", "CVC", "both"]
dataset = datasets[2]
args_train = setup_train_args(dataset=dataset, data_root="./data_root/",batch_size=8)

In [48]:
def train_teacher(args):
    (
        device,
        train_dataloader,
        val_dataloader,
        Dice_loss,
        BCE_loss,
        perf,
        model,
        optimizer,
    ) = build(args)
    
    loss_storage = []
    # performance metric is dice score
    perf_storage = []
    if not os.path.exists("./Trained models"):
        os.makedirs("./Trained models")

    prev_best_test = None
    if args.lrs == "true":
        if args.lrs_min > 0:
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode="max", factor=0.5, min_lr=args.lrs_min, verbose=True
            )
        else:
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode="max", factor=0.5, verbose=True
            )
    for epoch in range(1, args.epochs + 1):
        try:
            loss = train_epoch(
                model, device, train_dataloader, optimizer, epoch, Dice_loss, BCE_loss
            )
            test_measure_mean, test_measure_std = test(
                model, device, val_dataloader, epoch, perf
            )
            # add means and loss for vis
            loss_storage.append(loss)
            perf_storage.append(test_measure_mean)
            
        except KeyboardInterrupt:
            print("Training interrupted by user")
            sys.exit(0)
        if args.lrs == "true":
            scheduler.step(test_measure_mean)
        if prev_best_test == None or test_measure_mean > prev_best_test:
            print("Saving...")
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict()
                    if args.mgpu is False
                    else model.module.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "loss": loss,
                    "test_measure_mean": test_measure_mean,
                    "test_measure_std": test_measure_std,
                },
                "Trained models/FCBFormer_" + args.dataset + ".pt",
            )
            prev_best_test = test_measure_mean
    return loss_storage, perf_storage


In [49]:
losses, perfs = train_teacher(args=args_train)

Train Epoch: 1 [1288/1290 (100.0%)]	Average loss: 0.475360	Time: 76.884017
Test  Epoch: 1 [161/161 (100.0%)]	Average performance: 0.867256	Time: 5.280753
Saving...
Train Epoch: 2 [1288/1290 (100.0%)]	Average loss: 0.299826	Time: 78.156420
Test  Epoch: 2 [161/161 (100.0%)]	Average performance: 0.855021	Time: 5.301836
Train Epoch: 3 [712/1290 (55.3%)]	Loss: 0.158347	Time: 43.858248Training interrupted by user


AttributeError: 'tuple' object has no attribute 'tb_frame'